In [1]:
# Initial imports.
import pandas as pd
import numpy as np
import psycopg2
from datetime import date, datetime, timedelta
from matplotlib import pyplot as plt
# from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import tree

/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# Establish connection to AWS
connection = psycopg2.connect(
    host = 'group-eight-db.cchjt3hyglcz.us-east-1.rds.amazonaws.com',
    port = 5432,
    user = 'root',
    password = 'group_8_drought_2021',
    database='group_8_db_1'
    )
cursor=connection.cursor()

In [3]:
# Load soil csv files from AWS
sql = """
SELECT *
FROM "soil_data"
"""
soil_df = pd.read_sql(sql, con=connection)
soil_df.head()

,fips,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,6001.0,37.648081,-121.913304,146,0.0224,0.1172,0.1629,0.1930,0.1547,0.2397,...,4.510372,2.302448,6.812820,1,1,1,1,1,1,1
1,6003.0,38.617610,-119.798999,2630,0.0001,0.0026,0.0141,0.0596,0.0960,0.3779,...,0.000000,0.000000,0.000000,3,2,1,1,1,1,1
2,6005.0,38.443550,-120.653856,719,0.0003,0.0072,0.0461,0.1396,0.1640,0.2758,...,3.641502,0.000000,3.641502,1,1,3,1,1,1,2
3,6007.0,39.665959,-121.601919,373,0.0023,0.0263,0.1898,0.2433,0.1571,0.2899,...,3.188647,0.438461,3.627107,1,1,1,1,1,1,1
4,6009.0,38.187844,-120.555115,499,0.0015,0.0200,0.1085,0.2526,0.2027,0.3175,...,1.542023,2.162505,3.704528,1,1,2,1,1,1,2


In [4]:
# Load weather csv files from AWS
sql = """
SELECT *
FROM "drought_time_series"
"""
weather_df = pd.read_sql(sql, con=connection)
weather_df.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,6001.0,2019-01-01,0.00,100.28,2.76,3.81,-4.31,-4.14,10.89,-2.22,...,2.27,2.67,3.80,1.70,2.10,4.26,7.30,2.52,4.78,1.4968
1,6001.0,2019-01-02,0.00,100.35,3.45,5.97,-1.30,-1.30,12.20,1.48,...,3.27,1.26,1.82,0.38,1.44,1.70,2.67,0.41,2.25,NaN
2,6001.0,2019-01-03,0.00,100.11,3.79,5.77,-0.03,-0.11,12.50,1.55,...,3.82,1.76,2.24,1.23,1.02,2.47,3.19,1.74,1.46,NaN
3,6001.0,2019-01-04,0.00,99.42,4.55,7.44,2.42,2.15,14.84,2.37,...,5.62,1.46,2.06,0.46,1.59,2.26,3.93,0.79,3.14,NaN
4,6001.0,2019-01-05,8.84,98.76,6.67,8.90,7.71,7.67,13.09,5.58,...,8.20,5.54,9.79,2.17,7.61,8.53,13.18,4.11,9.07,NaN


In [5]:
# Loading weather data from local drive
#weather_df = pd.read_csv("data/test_timeseries.csv")
#weather_df.head()

In [6]:
# Loading soil data from local drive
#soil_df = pd.read_csv("data/soil_data.csv")
#soil_df.head()

In [7]:
# Limit the weather data to Texas
texas_weather = weather_df.loc[weather_df["fips"] >= 48000]
texas_weather = texas_weather.loc[weather_df["fips"] <= 48999]

In [69]:
# Merge the Texas weather dataframe with the soil dataframe for each county
texas_drought = pd.merge(texas_weather, soil_df, on='fips')
texas_drought

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,48001.0,2019-01-01,0.13,100.79,5.50,6.65,5.21,5.11,10.88,2.71,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
1,48001.0,2019-01-02,41.48,100.66,5.36,5.57,4.85,4.82,6.63,4.40,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
2,48001.0,2019-01-03,11.94,100.13,5.19,5.10,4.30,4.25,7.71,0.55,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
3,48001.0,2019-01-04,0.00,100.20,5.02,5.91,3.84,3.55,12.91,-0.25,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
4,48001.0,2019-01-05,0.00,100.56,6.34,9.13,7.25,7.09,17.84,3.22,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948429,48507.0,2018-12-27,0.10,98.42,5.86,13.59,5.85,5.62,21.99,7.47,...,0.000000,8.432994,8.432994,1,1,2,2,1,1,3
1948430,48507.0,2018-12-28,0.01,99.07,5.15,9.91,4.10,4.01,16.93,4.96,...,0.000000,8.432994,8.432994,1,1,2,2,1,1,3
1948431,48507.0,2018-12-29,0.01,99.63,4.40,7.99,1.96,1.95,10.21,6.43,...,0.000000,8.432994,8.432994,1,1,2,2,1,1,3
1948432,48507.0,2018-12-30,1.33,99.14,4.63,5.98,2.60,2.58,7.57,3.36,...,0.000000,8.432994,8.432994,1,1,2,2,1,1,3


In [70]:
# Remove unnecessary data
# Drop redundant weather data
texas_drought = texas_drought.drop(['T2MDEW', 'T2MWET', 'T2M_RANGE', 'WS10M_RANGE', 'WS50M_RANGE'], axis=1)
# Drop unnecessary soil data
texas_drought = texas_drought.drop(['lat', 'lon'], axis=1)

In [71]:
# Address NaN scores
# Drop NaN drought scores
#texas_drought.dropna(inplace=True)
# Set NaN drought scores to the most recent
texas_drought.fillna(method='ffill', inplace=True)
texas_drought.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2M_MAX,T2M_MIN,TS,WS10M,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,48001.0,2019-01-01,0.13,100.79,5.50,6.65,10.88,2.71,6.66,4.22,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
1,48001.0,2019-01-02,41.48,100.66,5.36,5.57,6.63,4.40,5.58,3.69,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
2,48001.0,2019-01-03,11.94,100.13,5.19,5.10,7.71,0.55,5.02,3.17,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
3,48001.0,2019-01-04,0.00,100.20,5.02,5.91,12.91,-0.25,4.85,3.11,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
4,48001.0,2019-01-05,0.00,100.56,6.34,9.13,17.84,3.22,7.96,2.88,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1


In [72]:
# Add scores for one week out to texas_drought dataframe (These are the scores we are trying to predict)
# Create new dataframe with dates and scores
week_later = texas_drought[['fips', 'date', 'score']]
week_later['prediction_date'] = week_later['date']

# Adjust the date forward seven days to get the week_before scores
time_delta = timedelta(-21)
texas_drought['date'] = pd.to_datetime(texas_drought['date'])
week_later['date'] = pd.to_datetime(week_later['date'])
week_later['prediction_date'] = week_later['date']
week_later['date'] = week_later['date'] + time_delta

# Merge the week_before scores into the texas_drought dataframe
texas_drought = pd.merge(texas_drought, week_later,  how='inner', left_on=['fips','date'], right_on = ['fips','date'])
texas_drought.head()

/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a co

,fips,date,PRECTOT,PS,QV2M,T2M,T2M_MAX,T2M_MIN,TS,WS10M,...,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,score_y,prediction_date
0,48001.0,2019-01-01,0.13,100.79,5.50,6.65,10.88,2.71,6.66,4.22,...,50.434738,3,2,1,1,1,1,1,0.0,2019-01-22
1,48001.0,2019-01-02,41.48,100.66,5.36,5.57,6.63,4.40,5.58,3.69,...,50.434738,3,2,1,1,1,1,1,0.0,2019-01-23
2,48001.0,2019-01-03,11.94,100.13,5.19,5.10,7.71,0.55,5.02,3.17,...,50.434738,3,2,1,1,1,1,1,0.0,2019-01-24
3,48001.0,2019-01-04,0.00,100.20,5.02,5.91,12.91,-0.25,4.85,3.11,...,50.434738,3,2,1,1,1,1,1,0.0,2019-01-25
4,48001.0,2019-01-05,0.00,100.56,6.34,9.13,17.84,3.22,7.96,2.88,...,50.434738,3,2,1,1,1,1,1,0.0,2019-01-26


In [73]:
# Address seasonality
# Add values from one year prior to texas_drought dataframe
# Create new dataframe with dates and scores
year_before = texas_drought[['fips', 'date', 'score_x']]

# Adjust the date forward seven days to get the week_before scores
# This is 51 weeks before the current date, which is 52 weeks before the predicted date, 364-x
time_delta = timedelta(+343)
year_before['date'] = pd.to_datetime(year_before['date'])
year_before['date'] = year_before['date'] + time_delta

# Merge the year_before scores into the texas_drought dataframe
texas_drought = pd.merge(texas_drought, year_before,  how='inner', left_on=['fips','date'], right_on = ['fips','date'])
texas_drought.head()

/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,fips,date,PRECTOT,PS,QV2M,T2M,T2M_MAX,T2M_MIN,TS,WS10M,...,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,score_y,prediction_date,score_x_y
0,48001.0,2019-01-01,0.13,100.79,5.50,6.65,10.88,2.71,6.66,4.22,...,3,2,1,1,1,1,1,0.0,2019-01-22,1.0
1,48001.0,2019-01-02,41.48,100.66,5.36,5.57,6.63,4.40,5.58,3.69,...,3,2,1,1,1,1,1,0.0,2019-01-23,1.0
2,48001.0,2019-01-03,11.94,100.13,5.19,5.10,7.71,0.55,5.02,3.17,...,3,2,1,1,1,1,1,0.0,2019-01-24,1.0
3,48001.0,2019-01-04,0.00,100.20,5.02,5.91,12.91,-0.25,4.85,3.11,...,3,2,1,1,1,1,1,0.0,2019-01-25,1.0
4,48001.0,2019-01-05,0.00,100.56,6.34,9.13,17.84,3.22,7.96,2.88,...,3,2,1,1,1,1,1,0.0,2019-01-26,1.0


In [74]:
# Clean up the 'score' columns
# Reorder the columns
texas_drought = texas_drought[['fips', 'date', 'PRECTOT', 'PS', 'QV2M', 'T2M', 'T2M_MAX', 'T2M_MIN',
       'TS', 'WS10M', 'WS10M_MAX', 'WS10M_MIN', 'WS50M', 'WS50M_MAX',
       'WS50M_MIN', 'elevation', 'slope1', 'slope2', 'slope3',
       'slope4', 'slope5', 'slope6', 'slope7', 'slope8', 'aspectN', 'aspectE',
       'aspectS', 'aspectW', 'aspectUnknown', 'WAT_LAND', 'NVG_LAND',
       'URB_LAND', 'GRS_LAND', 'FOR_LAND', 'CULTRF_LAND', 'CULTIR_LAND',
       'CULT_LAND', 'SQ1', 'SQ2', 'SQ3', 'SQ4', 'SQ5', 'SQ6', 'SQ7', 'score_x_x', 'prediction_date', 'score_y',
       'score_x_y']]
# Rename the score columns
texas_drought['score_to_be_predicted'] = texas_drought['score_y']
texas_drought['current_score'] = texas_drought['score_x_x']
texas_drought['score_year_before'] = texas_drought['score_x_y']
# Drop the old score columns
texas_drought.drop(['score_x_x', 'score_y', 'score_x_y'], axis=1, inplace=True)

In [75]:
texas_drought.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2M_MAX,T2M_MIN,TS,WS10M,...,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,prediction_date,score_to_be_predicted,current_score,score_year_before
0,48001.0,2019-01-01,0.13,100.79,5.50,6.65,10.88,2.71,6.66,4.22,...,2,1,1,1,1,1,2019-01-22,0.0,0.0,1.0
1,48001.0,2019-01-02,41.48,100.66,5.36,5.57,6.63,4.40,5.58,3.69,...,2,1,1,1,1,1,2019-01-23,0.0,0.0,1.0
2,48001.0,2019-01-03,11.94,100.13,5.19,5.10,7.71,0.55,5.02,3.17,...,2,1,1,1,1,1,2019-01-24,0.0,0.0,1.0
3,48001.0,2019-01-04,0.00,100.20,5.02,5.91,12.91,-0.25,4.85,3.11,...,2,1,1,1,1,1,2019-01-25,0.0,0.0,1.0
4,48001.0,2019-01-05,0.00,100.56,6.34,9.13,17.84,3.22,7.96,2.88,...,2,1,1,1,1,1,2019-01-26,0.0,0.0,1.0


In [76]:
# Split the training and testing data
train_enddate = datetime(year=2016, month=12, day=31)
Train = texas_drought.loc[texas_drought["date"] <= train_enddate]
Test = texas_drought.loc[texas_drought["date"] > train_enddate]

In [77]:
# Define the training features set.
X_train = Train.copy()
X_train = X_train.drop("score_to_be_predicted", axis=1)
X_train = X_train.drop("date", axis=1)
X_train = X_train.drop("prediction_date", axis=1)
X_train = X_train.drop("fips", axis=1)
X_train

,PRECTOT,PS,QV2M,T2M,T2M_MAX,T2M_MIN,TS,WS10M,WS10M_MAX,WS10M_MIN,...,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,current_score,score_year_before
710,0.04,101.58,4.05,3.56,10.44,-1.59,3.67,0.89,2.23,0.21,...,50.434738,3,2,1,1,1,1,1,0.3781,3.000
711,0.02,100.35,7.70,10.78,17.64,4.37,10.71,3.50,4.94,2.25,...,50.434738,3,2,1,1,1,1,1,0.0000,2.000
712,0.01,100.12,10.27,15.15,21.38,10.08,14.87,4.16,5.78,3.15,...,50.434738,3,2,1,1,1,1,1,0.0000,2.000
713,0.78,100.43,7.20,9.11,18.02,-4.44,9.07,6.81,9.47,4.58,...,50.434738,3,2,1,1,1,1,1,0.0000,2.000
714,7.42,101.42,2.12,-2.54,-0.01,-5.58,-2.10,4.68,6.93,2.19,...,50.434738,3,2,1,1,1,1,1,0.0000,2.909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855243,0.19,99.52,14.05,21.24,25.87,18.44,21.27,2.30,3.30,1.49,...,8.432994,1,1,2,2,1,1,3,0.0000,0.000
1855244,0.01,99.28,12.97,20.93,27.40,16.21,20.85,2.39,3.73,1.23,...,8.432994,1,1,2,2,1,1,3,0.0000,0.000
1855245,0.02,100.12,6.03,15.00,18.37,10.41,14.52,5.64,10.39,0.74,...,8.432994,1,1,2,2,1,1,3,0.0000,0.000
1855246,0.00,99.97,4.29,11.03,14.50,8.84,11.22,3.02,4.02,2.21,...,8.432994,1,1,2,2,1,1,3,0.0000,0.000


In [78]:
# Define the testing features set.
X_test = Test.copy()
X_test = X_test.drop("score_to_be_predicted", axis=1)
X_test = X_test.drop("date", axis=1)
X_test = X_test.drop("prediction_date", axis=1)
X_test = X_test.drop("fips", axis=1)
X_test

,PRECTOT,PS,QV2M,T2M,T2M_MAX,T2M_MIN,TS,WS10M,WS10M_MAX,WS10M_MIN,...,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,current_score,score_year_before
0,0.13,100.79,5.50,6.65,10.88,2.71,6.66,4.22,5.45,2.10,...,50.434738,3,2,1,1,1,1,1,0.0,1.0000
1,41.48,100.66,5.36,5.57,6.63,4.40,5.58,3.69,5.10,2.44,...,50.434738,3,2,1,1,1,1,1,0.0,1.0000
2,11.94,100.13,5.19,5.10,7.71,0.55,5.02,3.17,4.90,1.73,...,50.434738,3,2,1,1,1,1,1,0.0,1.0000
3,0.00,100.20,5.02,5.91,12.91,-0.25,4.85,3.11,5.07,1.21,...,50.434738,3,2,1,1,1,1,1,0.0,1.0000
4,0.00,100.56,6.34,9.13,17.84,3.22,7.96,2.88,3.68,1.63,...,50.434738,3,2,1,1,1,1,1,0.0,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855973,0.10,98.42,5.86,13.59,21.99,7.47,13.13,4.32,6.98,2.31,...,8.432994,1,1,2,2,1,1,3,0.0,0.2175
1855974,0.01,99.07,5.15,9.91,16.93,4.96,10.38,4.44,6.15,2.77,...,8.432994,1,1,2,2,1,1,3,0.0,0.2175
1855975,0.01,99.63,4.40,7.99,10.21,6.43,8.15,4.12,5.28,2.65,...,8.432994,1,1,2,2,1,1,3,0.0,0.2175
1855976,1.33,99.14,4.63,5.98,7.57,3.36,6.48,1.61,3.35,0.27,...,8.432994,1,1,2,2,1,1,3,0.0,0.2175


In [79]:
# Define the training target
y_train = Train["score_to_be_predicted"].ravel()
y_train[:5]

array([0., 0., 0., 0., 0.])

In [80]:
# Define the testing target
y_test = Test["score_to_be_predicted"].ravel()
y_test[:5]

array([0., 0., 0., 0., 0.])

In [81]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [82]:
# Create a random forest Regressor.
rf_model = RandomForestRegressor(n_estimators=128, max_depth=25, random_state=1) 

In [83]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

KeyboardInterrupt: 

In [67]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [68]:
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
mape = np.mean(np.abs((y_test - predictions) / np.abs(y_test)))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
print('Accuracy:', round(100*(1 - mape), 2))

Mean Absolute Error (MAE): 0.5485971341867966
Mean Squared Error (MSE): 0.5885039872275839
Root Mean Squared Error (RMSE): 0.767140135325733
Mean Absolute Percentage Error (MAPE): inf
Accuracy: -inf


/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.


In [27]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([5.10631618e-03, 1.99648304e-03, 2.83023790e-03, 1.30535398e-03,
       2.32643637e-03, 1.86194452e-03, 1.87355413e-03, 9.80364655e-04,
       1.32393005e-03, 1.18418121e-03, 1.01451756e-03, 1.39808537e-03,
       1.19265989e-03, 9.87391285e-04, 4.82491676e-04, 4.10301234e-04,
       6.45388279e-04, 4.95555605e-04, 3.83422316e-04, 2.47858645e-04,
       8.64175706e-05, 2.80242083e-05, 5.94350691e-04, 5.42219926e-04,
       5.55229070e-04, 6.47972148e-04, 3.27225129e-04, 2.39765170e-04,
       2.41473009e-04, 9.33985568e-04, 9.79704574e-04, 8.41906184e-04,
       5.82676090e-04, 8.42926513e-04, 6.92568283e-04, 7.40405643e-05,
       3.45660779e-05, 9.04068872e-05, 9.71211620e-05, 1.09331279e-05,
       3.31476102e-05, 1.60513099e-04, 9.60853168e-01, 2.46318553e-03])

In [30]:
# Visualize the Random Forest model
# (Need to reduce max_depth of rf_model in order to create useful visualization)

#fig = plt.figure(figsize=(15, 10))
#plot_tree(rf_model.estimators_[0], 
#          filled=True, impurity=True, 
#          rounded=True)